In [4]:
#pip install folium

SyntaxError: invalid syntax (968674204.py, line 1)

In [5]:
pip install polyline

Note: you may need to restart the kernel to use updated packages.


In [6]:
# import modules
import os
import requests
import urllib3
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import folium
import polyline
import base64
from tqdm import tqdm
# disable warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [60]:
# set strava variables
client_id = '119993'
client_secret = '67840b78f22679aab9989426c78b87b597a102de'
refresh_token = 'ec624131076c98a1d017f6fe24047eff07fdb52a'

https://www.strava.com/oauth/authorize?client_id=119993&redirect_uri=http://localhost&response_type=code&scope=activity:read_all



#http://localhost/?state=&code=f07a3664e7132519f4d62452a04d67dc0aa0cfc5&scope=read,activity:read_all
#http://localhost/?state=&code=7d25d089b532a3c14d5976b6110db66f49d02ae5&scope=read,activity:read_all

151b909d42918c5e8d066df90a82078502962f92

https://www.strava.com/oauth/token?client_id=119993&client_secret=67840b78f22679aab9989426c78b87b597a102de&code=151b909d42918c5e8d066df90a82078502962f92&grant_type=authorization_code

In [61]:
# define function to get a new access token
def get_access_token(client_id, client_secret, refresh_token):
 
   oauth_url = 'https://www.strava.com/oauth/token'
 
   payload = {
       'client_id': client_id, 
       'client_secret': client_secret, 
       'refresh_token': refresh_token, 
       'grant_type': 'refresh_token', 
       'f': 'json', 
   }
 
   r = requests.post(oauth_url, data=payload, verify=False)
 
   access_token = r.json()['access_token']
   return access_token
# get new access token
access_token = get_access_token(client_id, client_secret, refresh_token)
print (access_token)

9a986e65d750dbd304ef5d3ed8c3715f266cace3


In [62]:
# define function to get your strava data
def get_data(access_token, per_page=200, page=1):
 
   activities_url = 'https://www.strava.com/api/v3/athlete/activities'
   headers = {'Authorization': 'Bearer ' + access_token}
   params = {'per_page': per_page, 'page': page}
   
   data = requests.get(
       activities_url, 
       headers=headers, 
       params=params
   ).json()
 
   return data
# get you strava data
max_number_of_pages = 10
data = list()
for page_number in tqdm(range(1, max_number_of_pages + 1)):
    page_data = get_data(access_token, page=page_number)
    if page_data == []:
        break
    data.append(page_data)

 10%|████▌                                         | 1/10 [00:00<00:08,  1.07it/s]


In [51]:
activities_url = 'https://www.strava.com/api/v3/athlete/activities'
headers = {'Authorization': 'Bearer ' + access_token}
params = {'per_page': 200, 'page': page}
data = requests.get(
       activities_url, 
       headers=headers, 
       params=params
   ).json()

In [63]:
# data dictionaries
data_dictionaries = []
for page in data:
    data_dictionaries.extend(page)
# print number of activities
print('Number of activities downloaded: {}'.format(len(data_dictionaries)))

Number of activities downloaded: 3


In [64]:
print (data)

[[{'resource_state': 2, 'athlete': {'id': 128254009, 'resource_state': 1}, 'name': 'Morning Stand Up Paddling', 'distance': 8718.9, 'moving_time': 6643, 'elapsed_time': 7458, 'total_elevation_gain': 116.6, 'type': 'StandUpPaddling', 'sport_type': 'StandUpPaddling', 'id': 10566002522, 'start_date': '2024-01-15T20:12:44Z', 'start_date_local': '2024-01-15T10:12:44Z', 'timezone': '(GMT-10:00) Pacific/Honolulu', 'utc_offset': -36000.0, 'location_city': None, 'location_state': None, 'location_country': 'United States', 'achievement_count': 0, 'kudos_count': 0, 'comment_count': 0, 'athlete_count': 1, 'photo_count': 0, 'map': {'id': 'a10566002522', 'summary_polyline': 'gz{`C~qsa]VJV\\P@\\RRVb@LR^d@ZXNJRZNTVf@J^Td@FTLtB|CJTRTDLZTNTJFPBLJD\\NZRPL^PPJVLJJ`@^VDb@JPTPDNETBPDLZTFH@l@LVHh@b@d@D\\RXHXJJHb@LPRJFLGj@@TPdAMf@Ch@@ZVr@C`@Df@I\\AV?h@Nr@Ml@AVB`@Gl@Dz@[x@Gp@APF^Eb@Fz@EXAZLf@In@D\\TRDf@NXE|@FVJPMSIc@KEIWCBAEDZJVFl@R\\Bv@X^B`@JVPR?ZERB`@NRZPFNBVEVB^T^B^Rf@Ch@BX^l@Af@Jl@ANIVEh@Rj@@h@N^?b@Ph@G^B^

In [66]:
# normalize data
activities = pd.json_normalize(data_dictionaries)
# sample activities
activities[['name', 'distance', 'average_speed', 'moving_time']]\
.sample(3)

,name,distance,average_speed,moving_time
0,Morning Stand Up Paddling,8718.9,1.312,6643
1,Morning Stand Up Paddling,9926.5,1.482,6697
2,Lunch Stand Up Paddling,22810.6,3.289,6935


In [68]:
# add decoded summary polylines
activities['map.polyline'] = activities['map.summary_polyline'].apply(polyline.decode)

In [70]:
# convert data types
activities.loc[:, 'start_date'] = pd.to_datetime(activities['start_date']).dt.tz_localize(None)
activities.loc[:, 'start_date_local'] = pd.to_datetime(activities['start_date_local']).dt.tz_localize(None)
# convert values
activities.loc[:, 'distance'] /= 1000 # convert from m to km
activities.loc[:, 'average_speed'] *= 3.6 # convert from m/s to km/h
activities.loc[:, 'max_speed'] *= 3.6 # convert from m/s to km/h
# set index
activities.set_index('start_date_local', inplace=True)
# drop columns
activities.drop(
    [
        'map.summary_polyline', 
        'resource_state',
        'external_id', 
        'upload_id', 
        'location_city', 
        'location_state', 
        'has_kudoed', 
        'start_date', 
        'athlete.resource_state', 
        'utc_offset', 
        'map.resource_state', 
        'athlete.id', 
        'visibility', 
        'heartrate_opt_out', 
        'upload_id_str', 
        'from_accepted_tag', 
        'map.id', 
        'manual', 
        'private', 
        'flagged', 
    ], 
    axis=1, 
    inplace=True
)

In [77]:
# Save the map as an HTML file
m.save('interactive_map.html')

In [79]:
paddles = activities.iloc[:2, :]

# Calculate centroid for the first activity
centroid_first_activity = [
    np.mean([coord[0] for coord in paddles.iloc[0]['map.polyline']]), 
    np.mean([coord[1] for coord in paddles.iloc[0]['map.polyline']])
]

# Create a map with the initial location set to the centroid of the first activity
m = folium.Map(location=centroid_first_activity, zoom_start=10)

# Iterate over selected activities and plot on the map
for index, paddle in paddles.iterrows():
    # Calculate centroid
    centroid = [
        np.mean([coord[0] for coord in paddle['map.polyline']]), 
        np.mean([coord[1] for coord in paddle['map.polyline']])
    ]
    
    # Plot ride on map
    folium.PolyLine(paddle['map.polyline'], color='red').add_to(m)

# Display the map
display(m)
    # Save the map as an HTML file
m.save('interactive_map.html')